Import all modules you'll need. if you get an error like : no module "module_name" found; just run "pip install module_name" in empty cell.

In [2]:
%matplotlib Qt
%load_ext autoreload 
# to automaticaly reload all the modules that have been modified
%autoreload 2 
import os
from glob import glob
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from scipy import interpolate
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, ShuffleSplit
from sklearn import svm, kernel_ridge, metrics, preprocessing
from PIL import Image, ExifTags, ImageFile
import visualize as vis
import preprocessing as pp
import processing as proc
import calculate as cc
import pandas as pd
import matplotlib as mpl
import tensorflow as tf
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import correction as corr
from read_WDF import read_WDF, get_exif
import warnings
warnings.filterwarnings('ignore')

### Reading

In os.chdir, put the enplacement of your .wdf file and run the cell to get all your wdf files.

In [3]:
os.chdir('C:/Users/raoul.missodey/Desktop/Code_python')
files = glob('*.wdf')
for i, j in enumerate(files):
    print(i, ') ', j)

0 )  10nA_5h_obj10.wdf
1 )  800C_355nm.wdf
2 )  bida-a_pt1.wdf
3 )  bida-a_pt2.wdf
4 )  bida-a_pt3.wdf
5 )  bida-a_pt4.wdf
6 )  bida-a_pt5.wdf
7 )  B_carto_rayon_633nm_50%.wdf
8 )  Ca675A325_514nm.wdf
9 )  carto_10238_20x_HD1.wdf
10 )  carto_10238_20x_tr.wdf
11 )  carto_10291.wdf
12 )  carto_50x-bacteries-stramline.wdf
13 )  Carto_plaque_M3C0_7x7cm.wdf
14 )  carto_Z0_CR_NF3_curve fitting.wdf
15 )  cit-ACP1_chauffe_25-400°C_488nm_1-34sp.wdf
16 )  depth_Raoul.wdf
17 )  line.wdf
18 )  mapping.wdf
19 )  nax31_C_zone3_carto100x.wdf
20 )  ref-big-carto_633nm-50�-1_CCR.wdf
21 )  ref-big-carto_633nm-50�-1_CCR_Copy1.wdf
22 )  SBN4_ech3_poli-ethe_carto-ponctuelle.wdf
23 )  silice_LR.wdf
24 )  Spectre_M3C0_514nm.wdf
25 )  Streamline.wdf
26 )  test.wdf
27 )  UO2 vierge LD2600 633 VV_point.wdf
28 )  UO2 vierge LD2600 633 VV_point_1st_try.wdf
29 )  UO2 vierge LD2600 633 VV_point_Copy1.wdf
30 )  UO2 vierge LD2600 633 VV_point_Copy2nd.wdf
31 )  UO2 vierge LD2600 633 VV_point_Copy3.wdf
32 )  UO2 vierge

In [4]:
filename = files[37] # choose the number of .wdf file you want to use.
# load the file
da, img = read_WDF(filename, verbose=0) # verbose = 1 or true to have all information about your file.

#### Baseline Substraction

Two options to substract the baseline 

#### 1st option

In [ ]:
#  substract baseline : you have to modify lambda_value(smoothing parameter), p_value(the penalizing weighting factor)
# and lambda1_value(the smoothing parameter for the first derivative of the residual).
check_bline = vis.FindBaseline(da,sigma=da.shifts.data)

In [ ]:
# run to see chosen values
print(f"p: {check_bline.p_val:.2e}, lam: {check_bline.lam_val:.2e},lam1: {check_bline.lam1_val:.2e} ")
# calculate the basline for all spectrum
baseline = cc.baseline_ials(da, p=check_bline.p_val, lam=check_bline.lam_val, lam1= check_bline.lam1_val)
# substraction
db = da.copy()
db.values = db.values - baseline

#### 2nd option

In [5]:
check_bline = vis.FindBaseline1(da,sigma=da.shifts.data)

In [7]:
print(f"lam: {check_bline.lam_val:.2e}")
# calculate the basline for all spectrum
baseline = cc.baseline_arpls(da, lam=check_bline.lam_val)
# substraction
db = da.copy()
db.values = db.values - baseline

lam: 6.88e+06


### AutoEncoder for cosmic ray removal


In [9]:
# your firt output dimension must be less than the lenght of the spectra (or the total number of wavenumber)

class AutoEncoder(Model):
  def __init__(self):
    super(AutoEncoder, self).__init__()
    self.encoder = tf.keras.Sequential([
                  tf.keras.layers.Dense(256, activation="relu"),
                  tf.keras.layers.Dense(128, activation="relu"),
                  tf.keras.layers.Dense(64, activation="relu"),
                  tf.keras.layers.Dense(32, activation="relu"),
                  tf.keras.layers.Dense(16, activation="relu"),
                  tf.keras.layers.Dense(8, activation="relu")
              ])
    self.decoder = tf.keras.Sequential([
                  tf.keras.layers.Dense(16, activation="relu"),
                  tf.keras.layers.Dense(32, activation="relu"),
                  tf.keras.layers.Dense(64, activation="relu"),
                  tf.keras.layers.Dense(128, activation="relu"),
                  tf.keras.layers.Dense(256, activation="relu"),
                  tf.keras.layers.Dense(393, activation="sigmoid") # 391 is here the length of each spectra in my data
              ])
  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [10]:
X = db.data.copy() # put all intensity (after baseline substraction) in X

In [11]:
# split X into train and test
# test_size = 0.2 means 80% for training and 20% for test
X_train, X_test = train_test_split(X, test_size=0.2, random_state=228) 

In [12]:
# normalize the train and test to use it as validation data when training the neural network
# choose normalize method you want to apply : min_max, l1, l2, max, area, robust_scale 
#dm.values -= np.min(dm.values, axis=-1, keepdims=True)
Xn_train = pp.normalize(X_train, method="min_max")
Xn_test = pp.normalize(X_test, method="min_max")
Xn = pp.normalize(X, method="min_max")

In [14]:
# run this cell to train the model
#the first parameter is the training data
#the second parameter is the answer (here you can put the same number of train data but without cosmic ray to have a good model)
# epochs : nomber of time when the train data passes through the network 
# batch_size : number of spectra to use at each iteration
model = AutoEncoder()
#early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, mode="min")
model.compile(optimizer='adam', loss="mae")
history = model.fit(X_train, X_train, epochs=100, batch_size=64,
                    validation_data=(Xn_train, Xn_train),
                    shuffle=False
                   # callbacks=[early_stopping]
                    )

Epoch 1/100
99/99 [==============================] - 1s 5ms/step - loss: 0.0922 - val_loss: 0.0566
Epoch 2/100
99/99 [==============================] - 0s 4ms/step - loss: 0.0522 - val_loss: 0.0327
Epoch 3/100
99/99 [==============================] - 0s 4ms/step - loss: 0.0310 - val_loss: 0.0306
Epoch 4/100
99/99 [==============================] - 0s 4ms/step - loss: 0.0299 - val_loss: 0.0296
Epoch 5/100
99/99 [==============================] - 0s 4ms/step - loss: 0.0295 - val_loss: 0.0292
Epoch 6/100
99/99 [==============================] - 0s 4ms/step - loss: 0.0292 - val_loss: 0.0290
Epoch 7/100
99/99 [==============================] - 0s 4ms/step - loss: 0.0286 - val_loss: 0.0283
Epoch 8/100
99/99 [==============================] - 0s 4ms/step - loss: 0.0284 - val_loss: 0.0286
Epoch 9/100
99/99 [==============================] - 0s 4ms/step - loss: 0.0283 - val_loss: 0.0288
Epoch 10/100
99/99 [==============================] - 0s 4ms/step - loss: 0.0282 - val_loss: 0.0280
Epoch 11/

In [15]:
# test the neural network
encoder_out = model.encoder(Xn_test).numpy() #8 unit representation of data
decoder_out = model.decoder(encoder_out).numpy()

In [16]:
# plot the result
_s = np.stack((Xn_test,decoder_out), axis=-1)
show_result = vis.ShowSpectra(_s, labels=["test_spectra", "decoder"],sigma = da.shifts.data)

In [17]:
# plot loss curve
fig,ax = plt.subplots()
ax.plot(history.history['loss'])
ax.plot(history.history['val_loss'])
ax.set_xlabel('Epochs')
ax.set_ylabel('mae Loss')
ax.legend(['loss', 'val_loss'])

In [18]:
# apply it for all the data
Y = model.predict(Xn)

248/248 [==============================] - 0s 997us/step


In [19]:
# plot the result
_s = np.stack((Xn,Y), axis=-1)
show_result = vis.ShowSpectra(_s, labels=["original spectrum", "decoder"],sigma = da.shifts.data)

In [20]:
# save the result
db1 = db.copy()
db1.values = Y
db1.values -= np.min(db1.values, axis=-1, keepdims=True)
dn = pp.normalize(db1, method="min_max")

### Processing

#### PCA

In [22]:
# advanced svd : click inside the score image to have the related spectra, 
# change the feature_range to normalize the related spectra
advanced_svd = vis.SVD(dn, n_components=11, visualize_clean=1, visualize_components=1,
                       visualize_err=1,visualize_var=1,feature_range=(-0.075,0.075))
dsvd ,var,scores,loadings = advanced_svd.svd_element(dn)

In [ ]:
reconstruction = model.predict(Xn_test)
train_loss = tf.keras.losses.mae(reconstruction,Xn_test)
Train_loss = pd.Series(train_loss)
fig,ax = plt.subplots()
a,b,c = ax.hist(Train_loss,bins='auto')

In [20]:
fig,ax = plt.subplots()
a,b,c = ax.hist(Train_loss,bins='auto')

In [61]:
threshold = np.mean(train_loss) + 2*np.std(train_loss)
threshold

0.0343474569769878

In [65]:
preds = tf.math.less(Train_loss , threshold)
tf.math.count_nonzero(preds)

<tf.Tensor: shape=(), dtype=int64, numpy=1555>